In [1]:
#Import modules
import requests
import json
import pandas as pd
from config import api_key
import matplotlib.pyplot as plt
import numpy as np
import math
from scipy import stats

In [2]:
#URL and api key variables set
url_gold = "https://www.alphavantage.co/query?function=TIME_SERIES_DAILY_ADJUSTED&symbol=GOLD&interval=5min&apikey=api_key&outputsize=full"
url_oil = "https://www.alphavantage.co/query?function=TIME_SERIES_DAILY_ADJUSTED&symbol=OIL&interval=5min&apikey=api_key&outputsize=full" #"https://www.alphavantage.co/query?function=WTI&interval=daily"
url_btc = "https://www.alphavantage.co/query?function=DIGITAL_CURRENCY_DAILY&symbol=BTC&market=CNY&apikey=api_key"
url_sphq = "https://www.alphavantage.co/query?function=TIME_SERIES_DAILY_ADJUSTED&symbol=SPHQ&interval=5min&apikey=api_key&outputsize=full"

api_key = "&apikey=" + api_key

# SPHQ Data

In [3]:
#Data pull and pretty print with json
response_sphq = requests.get(url_sphq + api_key).json()
# print(json.dumps(response_sphq, indent=4, sort_keys=True))

In [4]:
#Data pull and create dataframe
sphq_data = response_sphq["Time Series (Daily)"]
sphq_df = pd.DataFrame(sphq_data).transpose()

sphq_df.head()

,1. open,2. high,3. low,4. close,5. adjusted close,6. volume,7. dividend amount,8. split coefficient
2023-04-06,47.16,47.38,46.99,47.31,47.31,328408,0.0000,1.0
2023-04-05,47.3,47.38,47.09,47.35,47.35,640077,0.0000,1.0
2023-04-04,47.82,47.84,47.23,47.36,47.36,738854,0.0000,1.0
2023-04-03,47.44,47.75,47.4,47.73,47.73,400099,0.0000,1.0
2023-03-31,46.89,47.36,46.8665,47.32,47.32,484691,0.0000,1.0


In [5]:
#Selecting coulmns for dataframe
organized_sphq_df = sphq_df[["1. open", "2. high", "3. low", "4. close" , "6. volume"]]
organized_sphq_df.head()

,1. open,2. high,3. low,4. close,6. volume
2023-04-06,47.16,47.38,46.99,47.31,328408
2023-04-05,47.3,47.38,47.09,47.35,640077
2023-04-04,47.82,47.84,47.23,47.36,738854
2023-04-03,47.44,47.75,47.4,47.73,400099
2023-03-31,46.89,47.36,46.8665,47.32,484691


In [6]:
#Relabeling columns without numbers
clean_sphq_df = organized_sphq_df.rename(columns={"1. open": "open", "2. high": "high", "3. low": "low" , "4. close": "close" , "6. volume": "volume"})
clean_sphq_df = clean_sphq_df.astype(float)
clean_sphq_df.head()

,open,high,low,close,volume
2023-04-06,47.16,47.38,46.9900,47.31,328408.0
2023-04-05,47.30,47.38,47.0900,47.35,640077.0
2023-04-04,47.82,47.84,47.2300,47.36,738854.0
2023-04-03,47.44,47.75,47.4000,47.73,400099.0
2023-03-31,46.89,47.36,46.8665,47.32,484691.0


In [7]:
#Calculating the daily change percent and creating a column for it 
daily_change_sphq = (100 * (clean_sphq_df["close"] - clean_sphq_df["open"] ) ) / clean_sphq_df["open"]
clean_sphq_df["daily change (%)"] = daily_change_sphq
clean_sphq_df.head()

,open,high,low,close,volume,daily change (%)
2023-04-06,47.16,47.38,46.9900,47.31,328408.0,0.318066
2023-04-05,47.30,47.38,47.0900,47.35,640077.0,0.105708
2023-04-04,47.82,47.84,47.2300,47.36,738854.0,-0.961941
2023-04-03,47.44,47.75,47.4000,47.73,400099.0,0.611298
2023-03-31,46.89,47.36,46.8665,47.32,484691.0,0.917040


In [8]:
#Create column for ticker symbol and organize order of columns
clean_sphq_df["symbol"] = "SPHQ"
final_sphq_df = clean_sphq_df[["symbol", "open", "high", "low", "close","daily change (%)"]]
#Remove dates from the index to a column in dataframe
final_sphq_df2 = final_sphq_df.reset_index()
#Rename column header from index to date
final_sphq_df3 = final_sphq_df2.rename(columns = {'index' : 'date'})
#Organize data from oldest date to newest
final_sphq_df4 = final_sphq_df3.sort_values('date')
final_sphq_df4.head()

,date,symbol,open,high,low,close,daily change (%)
4361,2005-12-06,SPHQ,15.47,15.69,15.470,15.60,0.840336
4360,2005-12-07,SPHQ,15.60,15.62,15.420,15.44,-1.025641
4359,2005-12-08,SPHQ,15.45,15.58,15.439,15.52,0.453074
4358,2005-12-09,SPHQ,15.59,15.74,15.470,15.57,-0.128287
4357,2005-12-12,SPHQ,15.75,15.80,15.590,15.67,-0.507937


In [9]:
#Create csv file for sphq dataframe
final_sphq_df4.to_csv("Resources/sphq_csv.csv", index=False)

# Gold Data

In [10]:
#API data request for gold data
response_gold = requests.get(url_gold + api_key).json()
# print(json.dumps(response_gold, indent=4, sort_keys=True))

In [11]:
#Data pull request for gold data "Time Series (Daily)"
gold_data = response_gold["Time Series (Daily)"]
gold_data["1999-11-01"]

{'1. open': '18.0',
 '2. high': '18.06',
 '3. low': '17.56',
 '4. close': '17.81',
 '5. adjusted close': '13.0256374447205',
 '6. volume': '1678800',
 '7. dividend amount': '0.0000',
 '8. split coefficient': '1.0'}

In [12]:
#Switch columns and row data
gold_df = pd.DataFrame(gold_data).transpose()

gold_df.head()

,1. open,2. high,3. low,4. close,5. adjusted close,6. volume,7. dividend amount,8. split coefficient
2023-04-06,19.55,19.74,19.395,19.73,19.73,17237020,0.0000,1.0
2023-04-05,19.85,20.05,19.46,19.68,19.68,25588361,0.0000,1.0
2023-04-04,18.78,19.72,18.74,19.62,19.62,28821147,0.0000,1.0
2023-04-03,18.67,19.03,18.48,18.79,18.79,15481362,0.0000,1.0
2023-03-31,18.66,18.75,18.38,18.57,18.57,15434455,0.0000,1.0


In [13]:
#Reorganize columns in gold dataframe
organized_gold_df = gold_df[["1. open", "2. high", "3. low", "4. close" , "6. volume"]]
organized_gold_df.head()

,1. open,2. high,3. low,4. close,6. volume
2023-04-06,19.55,19.74,19.395,19.73,17237020
2023-04-05,19.85,20.05,19.46,19.68,25588361
2023-04-04,18.78,19.72,18.74,19.62,28821147
2023-04-03,18.67,19.03,18.48,18.79,15481362
2023-03-31,18.66,18.75,18.38,18.57,15434455


In [14]:
#Relabeled columns for gold dataframe
clean_gold_df = organized_gold_df.rename(columns={"1. open": "open", "2. high": "high", "3. low": "low" , "4. close": "close" , "6. volume": "volume"})
clean_gold_df = clean_gold_df.astype(float)
clean_gold_df.head()

,open,high,low,close,volume
2023-04-06,19.55,19.74,19.395,19.73,17237020.0
2023-04-05,19.85,20.05,19.460,19.68,25588361.0
2023-04-04,18.78,19.72,18.740,19.62,28821147.0
2023-04-03,18.67,19.03,18.480,18.79,15481362.0
2023-03-31,18.66,18.75,18.380,18.57,15434455.0


In [15]:
#Calculating the daily change percent and creating a column for it 
daily_change_gold = (100 * (clean_gold_df["close"] - clean_gold_df["open"] ) ) / clean_gold_df["open"]
clean_gold_df["daily change (%)"] = daily_change_gold
clean_gold_df.head()

,open,high,low,close,volume,daily change (%)
2023-04-06,19.55,19.74,19.395,19.73,17237020.0,0.920716
2023-04-05,19.85,20.05,19.460,19.68,25588361.0,-0.856423
2023-04-04,18.78,19.72,18.740,19.62,28821147.0,4.472843
2023-04-03,18.67,19.03,18.480,18.79,15481362.0,0.642742
2023-03-31,18.66,18.75,18.380,18.57,15434455.0,-0.482315


In [16]:
#Create column with rows filled with "GOLD"
clean_gold_df["symbol"] = "GOLD"
#Reorganize the order of columns
final_gold_df = clean_gold_df[["symbol", "open", "high", "low", "close","daily change (%)"]]
#Change date from index to column in dataframe
final_gold_df2 = final_gold_df.reset_index()
#Rename index column to date
final_gold_df3 = final_gold_df2.rename(columns = {'index' : 'date'})
#Organize the data from oldest to newest 
final_gold_df4 = final_gold_df3.sort_values('date')
final_gold_df4.head()

,date,symbol,open,high,low,close,daily change (%)
5895,1999-11-01,GOLD,18.00,18.06,17.56,17.81,-1.055556
5894,1999-11-02,GOLD,17.81,18.25,17.63,18.25,2.470522
5893,1999-11-03,GOLD,18.19,18.25,17.75,17.88,-1.704233
5892,1999-11-04,GOLD,17.69,17.94,17.56,17.69,0.000000
5891,1999-11-05,GOLD,17.56,17.69,17.13,17.19,-2.107062


In [17]:
#Create CSV file for gold dateframe
final_gold_df4.to_csv("Resources/gold_csv.csv", index=False)

# Oil Data 

In [18]:
#API request for oil data
response_oil = requests.get(url_oil + api_key).json()
# print(json.dumps(response_oil, indent=4, sort_keys=True))

In [19]:
#Format data into dataframe, organize columns and rename columns
oil_data = response_oil["Time Series (Daily)"]
oil_df = pd.DataFrame(oil_data).transpose()
organized_oil_df = oil_df[["1. open", "2. high", "3. low", "4. close" , "6. volume"]]
clean_oil_df = organized_oil_df.rename(columns={"1. open": "open", "2. high": "high", "3. low": "low" , "4. close": "close" , "6. volume": "volume"})
clean_oil_df = clean_oil_df.astype(float)
clean_oil_df.head()

,open,high,low,close,volume
2023-04-06,30.58,30.5800,30.28,30.58,17546.0
2023-04-05,30.49,30.5608,30.18,30.49,24863.0
2023-04-04,30.90,30.9000,30.23,30.43,147521.0
2023-04-03,30.30,30.6400,30.20,30.50,68863.0
2023-03-31,28.56,28.9500,28.55,28.95,24544.0


In [20]:
#Calculating the daily change percent and creating a column for it 
daily_change_oil = (100 * (clean_oil_df["close"] - clean_oil_df["open"] ) ) / clean_oil_df["open"]
clean_oil_df["daily change (%)"] = daily_change_oil
clean_oil_df.head()

,open,high,low,close,volume,daily change (%)
2023-04-06,30.58,30.5800,30.28,30.58,17546.0,0.000000
2023-04-05,30.49,30.5608,30.18,30.49,24863.0,0.000000
2023-04-04,30.90,30.9000,30.23,30.43,147521.0,-1.521036
2023-04-03,30.30,30.6400,30.20,30.50,68863.0,0.660066
2023-03-31,28.56,28.9500,28.55,28.95,24544.0,1.365546


In [21]:
#Create column with rows filled with "OIL"
clean_oil_df["symbol"] = "OIL"
#Reorganize column order
final_oil_df = clean_oil_df[["symbol", "open", "high", "low", "close", "daily change (%)"]]
#Move dates from index to column in dataframe
final_oil_df2 = final_oil_df.reset_index()
#Rename column from index to date
final_oil_df3 = final_oil_df2.rename(columns = {'index' : 'date'})
#Sort dates from oldest to newest
final_oil_df4 = final_oil_df3.sort_values('date')
final_oil_df4.head()

,date,symbol,open,high,low,close,daily change (%)
3009,2011-04-21,OIL,49.94,50.35,49.90,50.35,0.820985
3008,2011-04-25,OIL,50.57,50.57,49.91,50.05,-1.028278
3007,2011-04-26,OIL,50.18,50.43,50.10,50.30,0.239139
3006,2011-04-27,OIL,50.35,50.85,49.80,50.81,0.913605
3005,2011-04-28,OIL,50.78,51.16,50.17,50.70,-0.157542


In [22]:
#Create CSV file for oil dataframe
final_oil_df4.to_csv("Resources/oil_csv.csv", index=False)

In [23]:
#Check for null values in oil dataframe
final_oil_df4.isnull().sum()

date                0
symbol              0
open                0
high                0
low                 0
close               0
daily change (%)    0
dtype: int64

# Bitcoin Data

In [24]:
#API request for bitcoin data
response_btc = requests.get(url_btc + api_key).json()
# print(json.dumps(response_btc, indent=4, sort_keys=True))

In [25]:
#Build data into dataframe for bitcoin
btc_data = response_btc["Time Series (Digital Currency Daily)"]
#change orientation of dataframe
btc_df = pd.DataFrame(btc_data).transpose()
#Specify which columns to use
organized_btc_df = btc_df[["1b. open (USD)", "2b. high (USD)", "3b. low (USD)", "4b. close (USD)", "5. volume"]]
#Rename columns 
clean_btc_df = organized_btc_df.rename(columns={"1b. open (USD)": "open", "2b. high (USD)": "high", "3b. low (USD)": "low" , "4b. close (USD)": "close" , "5. volume": "volume"})
clean_btc_df.head()

,open,high,low,close,volume
2023-04-07,28033.83000000,28051.58000000,27998.04000000,28050.81000000,498.52275000
2023-04-06,28170.01000000,28182.05000000,27711.00000000,28033.82000000,40118.94963000
2023-04-05,28165.47000000,28775.00000000,27805.10000000,28170.01000000,60737.64732000
2023-04-04,27800.00000000,28444.44000000,27662.79000000,28165.47000000,49722.55691000
2023-04-03,28171.87000000,28500.99000000,27200.24000000,27800.00000000,79180.01405000


In [26]:
#Adding symbol column ad fill with "BTC"
clean_btc_df["symbol"] = "BTC"
clean_btc_df.head()

,open,high,low,close,volume,symbol
2023-04-07,28033.83000000,28051.58000000,27998.04000000,28050.81000000,498.52275000,BTC
2023-04-06,28170.01000000,28182.05000000,27711.00000000,28033.82000000,40118.94963000,BTC
2023-04-05,28165.47000000,28775.00000000,27805.10000000,28170.01000000,60737.64732000,BTC
2023-04-04,27800.00000000,28444.44000000,27662.79000000,28165.47000000,49722.55691000,BTC
2023-04-03,28171.87000000,28500.99000000,27200.24000000,27800.00000000,79180.01405000,BTC


In [27]:
#Select columns to be used-remove symbol to change data type
final_btc_df = clean_btc_df[["open", "high", "low", "close"]]
#Change data type to float
final_btc_df = final_btc_df.astype(float)
#Change variable name for dataframe
new_final = final_btc_df


new_final.head()

,open,high,low,close
2023-04-07,28033.83,28051.58,27998.04,28050.81
2023-04-06,28170.01,28182.05,27711.00,28033.82
2023-04-05,28165.47,28775.00,27805.10,28170.01
2023-04-04,27800.00,28444.44,27662.79,28165.47
2023-04-03,28171.87,28500.99,27200.24,27800.00


# Bitcoin Raw Data

In [28]:
#Path for bitcoin source file from Kaggle
file = "Resources/bitcoin_raw.csv"

In [29]:
#Read file path for CSV and display dataframe
btc_orig_df = pd.read_csv(file)
#Change date column to the index
btc_orig_df = btc_orig_df.set_index("Date")
#Select columns for dataframe-remove currency to convert data type
btc_orig_df = btc_orig_df[['Closing Price (USD)', '24h Open (USD)' , '24h High (USD)' ,'24h Low (USD)']]
#Change data type to float
btc_orig_df = btc_orig_df.astype(float)

btc_orig_df.head()

,Closing Price (USD),24h Open (USD),24h High (USD),24h Low (USD)
Date,,,,
2013-10-01,123.65499,124.30466,124.75166,122.56349
2013-10-02,125.45500,123.65499,125.75850,123.63383
2013-10-03,108.58483,125.45500,125.66566,83.32833
2013-10-04,118.67466,108.58483,118.67500,107.05816
2013-10-05,121.33866,118.67466,121.93633,118.00566


In [30]:
#Rename columns
organized_btc_orig = btc_orig_df[["24h Open (USD)","24h High (USD)","24h Low (USD)", "Closing Price (USD)"]] 
# ^^Took out 'Currency' because it's not as one of the columns and 'Date' because it's the index.
clean_btc_orig = organized_btc_orig.astype(float)
# ^^ changed it to float.

#Renamed columns to match other dateframes
clean_btc_orig = clean_btc_orig.rename(columns={"Date": "date","Currency": "symbol","24h Open (USD)": "open",
                                                    "24h High (USD)": "high","24h Low (USD)": "low",
                                                    "Closing Price (USD)": "close"})

clean_btc_orig.head()

,open,high,low,close
Date,,,,
2013-10-01,124.30466,124.75166,122.56349,123.65499
2013-10-02,123.65499,125.75850,123.63383,125.45500
2013-10-03,125.45500,125.66566,83.32833,108.58483
2013-10-04,108.58483,118.67500,107.05816,118.67466
2013-10-05,118.67466,121.93633,118.00566,121.33866


# Bitcoin Combined

In [31]:
#Combine all bitcoin databases together
join_btc = pd.concat([new_final, clean_btc_orig], ignore_index=False) 
#Change date index to column
join_btc = join_btc.reset_index()
#REname column name from index to date
join_btc = join_btc.rename(columns={"index" : "date"})
#Organize date from oldest to newest dates
final_btc = join_btc.sort_values('date')
#Create column named symbol and fill with BTC
final_btc['symbol'] = 'BTC'
#Organize the columns for dataframe
final_join_btc = final_btc[['date','symbol', 'open', 'high', 'low', 'close']]

final_join_btc.head()

,date,symbol,open,high,low,close
1000,2013-10-01,BTC,124.30466,124.75166,122.56349,123.65499
1001,2013-10-02,BTC,123.65499,125.75850,123.63383,125.45500
1002,2013-10-03,BTC,125.45500,125.66566,83.32833,108.58483
1003,2013-10-04,BTC,108.58483,118.67500,107.05816,118.67466
1004,2013-10-05,BTC,118.67466,121.93633,118.00566,121.33866


In [32]:
#Calculate the daily change percent
daily_change_btc = (100 * (final_join_btc["close"] - final_join_btc["open"] ) ) / final_join_btc["open"]
#Create column name daily change percent
final_join_btc["daily change (%)"] = daily_change_btc

#Create CSV file for combined Bitcoin datasets
final_join_btc.to_csv("Resources/btc_joined.csv", index=False)

#Change date column to the index
final_join_btc2 = final_join_btc.set_index("date")
final_join_btc2.head()

C:\Users\alexp\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,symbol,open,high,low,close,daily change (%)
date,,,,,,
2013-10-01,BTC,124.30466,124.75166,122.56349,123.65499,-0.522643
2013-10-02,BTC,123.65499,125.75850,123.63383,125.45500,1.455671
2013-10-03,BTC,125.45500,125.66566,83.32833,108.58483,-13.447188
2013-10-04,BTC,108.58483,118.67500,107.05816,118.67466,9.292118
2013-10-05,BTC,118.67466,121.93633,118.00566,121.33866,2.244793
